In [1]:
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import requests  # External API
import os  # For accessing environment variables securely

# Load data and prepare user-item matrix
ratings = pd.read_csv("C:\\Users\\zahra\\Downloads\\Telegram Desktop\\ratings.csv")
movies = pd.read_csv("C:\\Users\\zahra\\Downloads\\Telegram Desktop\\movies.csv")
merged_data = pd.merge (ratings, movies, on = 'movieId', how='inner')

# Convert user-item matrix to a sparse matrix format
user_item_matrix = merged_data.pivot_table(index='userId', columns='title', values='rating').fillna(0)
user_item_matrix_sparse = csr_matrix(user_item_matrix.values)

# Compute user similarity
user_similarity = cosine_similarity(user_item_matrix_sparse)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Flask application
app = Flask(__name__)

# Movie Recommendation Function (based on collaborative filtering)
def recommend_movies(user_id, user_item_matrix, user_similarity_df, num_recommendations=5, top_n_similar_users=100):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).head(top_n_similar_users).index
    similar_user_ratings = user_item_matrix.loc[similar_users]
    weighted_ratings = similar_user_ratings.mean().sort_values(ascending=False)
    already_rated = user_item_matrix.loc[user_id]
    recommendations = weighted_ratings[~already_rated.index.isin(already_rated[already_rated > 0].index)]
    recommendations_df = pd.DataFrame(recommendations).reset_index()
    recommendations_df.columns = ['Movie Title', 'Predicted Rating']
    return recommendations_df.head(num_recommendations)

# TMDb API function to get additional recommendations
def external_api_recommendations(movie_title, api_key):
    try:
        url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={movie_title}"
        response = requests.get(url)
        response_data = response.json()
        
        # Get the movie's id for further recommendations
        if response_data['results']:
            movie_id = response_data['results'][0]['id']
            recommendation_url = f"https://api.themoviedb.org/3/movie/{movie_id}/recommendations?api_key={api_key}"
            recommendations_response = requests.get(recommendation_url)
            recommendations_data = recommendations_response.json()
            return [movie['title'] for movie in recommendations_data['results'][:5]]  # Return top 5 recommendations
        else:
            return []
    except Exception as e:
        print(f"Error fetching recommendations: {e}")
        return []

# API route to get recommendations
@app.route('/recommend', methods=['GET'])
def recommend():
    user_id = int(request.args.get('user_id'))
    num_recommendations = int(request.args.get('num_recommendations', 5))
    
    # Call the recommendation function (collaborative filtering)
    internal_recommendations = recommend_movies(user_id=user_id, user_item_matrix=user_item_matrix, user_similarity_df=user_similarity_df, num_recommendations=num_recommendations)
    
    # Use an external API (e.g., TMDb) for more recommendations based on the top movie
    top_movie = internal_recommendations.iloc[0]['Movie Title']
    tmdb_api_key = os.getenv("TMDB_API_KEY")  # Make sure to set your API key as an environment variable
    external_recommendations = external_api_recommendations(top_movie, tmdb_api_key)
    
    # Combine both sets of recommendations into one response
    combined_recommendations = {
        'internal_recommendations': internal_recommendations.to_dict(orient='records'),
        'external_recommendations': external_recommendations
    }
    
    # Return the recommendations as a JSON response
    return jsonify(combined_recommendations)

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\zahra\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
